In [0]:
drop schema adventureworks.silver cascade;
create schema if not exists adventureworks.silver;


In [0]:
%python
from pyspark.sql import functions as F

In [0]:
%python
base_path = 'adventureworks.bronze'
spark.sql(f"USE {base_path}")

In [0]:
%python
df_addresses = spark.table("address")
df_customer = spark.table("customer")
df_customer_address = spark.table("customer_address")

In [0]:
%python

df_customer = df_customer.withColumn("full_name",
    F.when(F.col("first_name").isNotNull() & F.col("last_name").isNotNull(),
         F.concat_ws(" ",
                  F.col("first_name"),
                  F.when(F.col("middle_name").isNotNull(), F.col("middle_name")),
                  F.col("last_name"))
    ).otherwise(None))
    
display(df_customer)

In [0]:
%python
df_addresses = df_addresses\
  .withColumn("address_line1",
              F.concat_ws(", ",
                          F.col("address_line1"),
                          F.when(F.col("address_line2").isNotNull(), F.col("address_line2"))))

display(df_addresses)

In [0]:
%python
df_main_address = df_customer_address.alias("ca").filter(F.col("ca.address_type") == "Main Office")\
    .join(df_addresses.alias("a"),
          F.col("ca.address_id") == F.col("a.address_id"),
          "left")\
    .select("ca.customer_id",
            F.col("a.address_line1").alias("main_address"),
            F.col("a.city").alias("main_city"),
            F.col("a.state_province").alias("main_state_province"),
            F.col("a.country_region").alias("main_country_region"),
            F.col("a.postal_code").alias("main_postal_code"))

display(df_main_address)

In [0]:
%python

df_shipping_address = df_customer_address.alias("ca").filter(F.col("ca.address_type") == "Shipping")\
    .join(df_addresses.alias("a"),
          F.col("ca.address_id") == F.col("a.address_id"),
          "left")\
    .select("ca.customer_id",
            "ca.address_id",
            F.col("a.address_line1").alias("shipping_address"),
            F.col("a.city").alias("shipping_city"),
            F.col("a.state_province").alias("shipping_state_province"),
            F.col("a.country_region").alias("shipping_country_region"),
            F.col("a.postal_code").alias("shipping_postal_code"))

display(df_shipping_address)

In [0]:
%python
df_customer_address = df_main_address.alias("ma")\
       .join(df_shipping_address.alias("sa"),
             F.col("ma.customer_id") == F.col("sa.customer_id"),
             "left")\
       .select(
               "ma.customer_id",
               
               "ma.main_address",
               "ma.main_city",
               "ma.main_state_province",
               "ma.main_country_region",
               "ma.main_postal_code",

               "sa.shipping_address",
               "sa.shipping_city",
               "sa.shipping_state_province",
               "sa.shipping_country_region",
               "sa.shipping_postal_code")\
       .withColumn("shipping_address",
                   F.when(F.col("shipping_address").isNotNull(),
                          F.col("shipping_address")).otherwise(F.col("main_address")))\
       .withColumn("shipping_city",
                   F.when(F.col("shipping_city").isNotNull(),
                          F.col("shipping_city")).otherwise(F.col("main_city")))\
       .withColumn("shipping_state_province",
                   F.when(F.col("shipping_state_province").isNotNull(),
                          F.col("shipping_state_province")).otherwise(F.col("main_state_province")))\
       .withColumn("shipping_country_region",
                   F.when(F.col("shipping_country_region").isNotNull(),
                          F.col("shipping_country_region")).otherwise(F.col("main_country_region")))\
       .withColumn("shipping_postal_code",
                   F.when(F.col("shipping_postal_code").isNotNull(),
                          F.col("shipping_postal_code")).otherwise(F.col("main_postal_code")))

display(df_customer_address)
       



In [0]:
%python
df_show = df_customer_address.filter((F.col("main_address")) != (F.col("shipping_address")))
display(df_show)

In [0]:
%python
df_show = df_customer_address.filter((F.col("main_address")) == (F.col("shipping_address")))
display(df_show)

In [0]:
%python
df_silver_customers =  df_customer.alias("c")\
    .join(df_customer_address.alias("ca"),
          F.col("c.customer_id") == F.col("ca.customer_id"),
          "left")\
    .select(
        #Identificadores
        "c.customer_id",

        #Name
        "c.first_name",
        "c.middle_name",
        "c.last_name",
        "c.full_name",
        "c.title",
        "c.suffix",

        #Contato
        "c.email_address",
        "c.phone",

        #Main Endereço
        "ca.main_address",
        "ca.main_city",
        "ca.main_state_province",
        "ca.main_country_region",
        "ca.main_postal_code",

        #shipping endereço
        "ca.shipping_address",
        "ca.shipping_city",
        "ca.shipping_state_province",
        "ca.shipping_country_region",
        "ca.shipping_postal_code",
        

        # Metadados
        "c.modified_date",
        
        # Validações
        F.when(F.col("c.email_address").isNotNull(), "VALID_EMAIL").otherwise("INVALID_EMAIL").alias("email_status"),
        F.when(F.col("c.full_name").isNotNull(), "VALID_NAME").otherwise("INVALID_NAME").alias("name_status"),
        F.when(F.col("ca.main_postal_code").isNotNull(), "HAS_ADDRESS").otherwise("MISSING_ADDRESS").alias("address_status"))

display(df_silver_customers)

In [0]:
%python
df_silver_customers = df_silver_customers\
        .withColumn("customer_id", F.col("customer_id").cast("int"))\
        .withColumn("modified_date", F.to_date(F.col("modified_date")))
display(df_silver_customers.limit(10))

In [0]:
%python
# COMMAND ----------

# Validações e Qualidade de Dados

validation_results = {}

# 1. Verificar duplicados por email
email_duplicates = df_silver_customers \
    .filter(F.col("email_address").isNotNull()) \
    .groupBy("email_address") \
    .count() \
    .filter(F.col("count") > 1) \
    .count()
validation_results["email_duplicates"] = email_duplicates

# 2. Verificar valores nulos em campos críticos
critical_nulls = df_silver_customers \
    .filter(F.col("full_name").isNull() | F.col("customer_id").isNull()) \
    .count()
validation_results["critical_nulls"] = critical_nulls

# 3. Verificar emails inválidos
invalid_emails = df_silver_customers \
    .filter(F.col("email_status") == "INVALID_EMAIL") \
    .count()
validation_results["invalid_emails"] = invalid_emails

# 4. Verificar formato de telefones
invalid_phones = df_silver_customers \
    .filter(
        (F.col("phone").isNotNull()) & 
        (~F.col("phone").rlike(r'^[\d\s\-\+\(\)]+$'))
    ) \
    .count()
validation_results["invalid_phones"] = invalid_phones

print("Resultados da Validação:")
for check, count in validation_results.items():
    status = "ok" if count == 0 else "not ok"
    print(f"   {status} {check}: {count}")
    print("")



In [0]:
%python
from pyspark.sql.window import Window

# Janela por email
w = Window.partitionBy("email_address")

df_silver_customers = (
    df_silver_customers
    .withColumn("email_count", F.count("*").over(w))
    .filter(
        ~(
            (F.col("email_count") > 1) &
            (F.col("main_address").isNull())
        )
    )
    .drop("email_count")
)


In [0]:
%python
# COMMAND ----------

# Validações e Qualidade de Dados

# 1. Verificar duplicados por email
email_duplicates = df_silver_customers \
    .filter(F.col("email_address").isNotNull()) \
    .groupBy("email_address") \
    .count() \
    .filter(F.col("count") > 1) \
    .count()


print(f"Email Duplicados: {email_duplicates}")




In [0]:
%python

silver_customers_path = 'adventureworks.silver.customers'

df_silver_customers.write.mode("overwrite").saveAsTable(silver_customers_path)

# Otimizar tabela
spark.sql("OPTIMIZE silver.customers")
spark.sql("ANALYZE TABLE silver.customers COMPUTE STATISTICS")    

In [0]:
select *
from adventureworks.silver.customers